# Shade-Optimized Pedestrian Routing to Transit
## University City, Philadelphia

**Author:** Kavana Raju  
**Course:** MUSA 5500 - Geospatial Data Science with Python

## Notebook 3: Time-of-Day & Seasonal Routing Analysis

This notebook analyzes how shade-optimized routing performance varies across:
- **Different times of day** (morning, midday, evening)
- **Different seasons** (summer, winter, spring, fall)
- **Multiple routes** (Penn Campus, Drexel, Clark Park to transit)

We compare routing efficiency, detour distances, and shade improvements across 8 scenarios.

### Setup and Imports

In [20]:
# Standard libraries
import os
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
import pandas as pd
import numpy as np

# Geospatial
import geopandas as gpd
import osmnx as ox
import networkx as nx

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Time/date
from datetime import datetime
import pytz

# Configure
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 10)

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


## Part 1: Load Enhanced Network with Solar Data

In [21]:
# Load network with solar shadow data
G = ox.load_graphml('data/processed/university_city_network_solar.graphml')
print(f"✓ Network loaded: {len(G.nodes):,} nodes, {len(G.edges):,} edges")

# Load SEPTA stops
septa_gdf = gpd.read_file('data/processed/septa_stops.geojson')
print(f"✓ SEPTA stops loaded: {len(septa_gdf)} stops")

# Load edges with solar data
edges_gdf = gpd.read_file('data/processed/network_edges_solar.geojson')
print(f"✓ Edges GeoDataFrame loaded: {len(edges_gdf):,} edges")

# Check what scenarios we have
scenario_cols = [col for col in edges_gdf.columns if col.startswith('shade_')]
scenarios = [col.replace('shade_', '') for col in scenario_cols]
print(f"\n✓ Found {len(scenarios)} time scenarios:")
for s in scenarios:
    print(f"  - {s}")

✓ Network loaded: 7,343 nodes, 23,486 edges
✓ SEPTA stops loaded: 60 stops
✓ Edges GeoDataFrame loaded: 23,486 edges

✓ Found 8 time scenarios:
  - summer_morning
  - summer_midday
  - summer_evening
  - winter_morning
  - winter_midday
  - winter_evening
  - spring_midday
  - fall_midday


## Part 2: Time-Specific Routing Function

In [27]:
def path_to_coords(path, graph):
    """
    Convert node path to lat/lon coordinates for mapping.
    """
    coords = []
    for node in path:
        y = graph.nodes[node]['y']
        x = graph.nodes[node]['x']
        coords.append((y, x))
    return coords

def calculate_time_specific_route(origin_coords, dest_stop_name, scenario_name,
                                   graph, septa_gdf, verbose=True):
    """
    Calculate routes using time-specific shade weights.
    
    FIXED: Properly handles MultiDiGraph edge access
    """
    lat, lon = origin_coords
    
    # Find nearest nodes
    origin_node = ox.nearest_nodes(graph, lon, lat)
    
    dest_stops = septa_gdf[septa_gdf['name'] == dest_stop_name]
    if len(dest_stops) == 0:
        if verbose:
            print(f"⚠️  Stop '{dest_stop_name}' not found")
        return None
    
    dest_stop = dest_stops.iloc[0]
    dest_node = ox.nearest_nodes(graph, dest_stop.geometry.x, dest_stop.geometry.y)
    
    results = {
        'scenario': scenario_name,
        'origin_lat': lat,
        'origin_lon': lon,
        'dest_name': dest_stop_name,
        'dest_lat': dest_stop.geometry.y,
        'dest_lon': dest_stop.geometry.x,
    }
    
    # Calculate SHORTEST route (using length)
    try:
        shortest_path = nx.shortest_path(graph, origin_node, dest_node, weight='length')
        
        # Calculate total length and weighted shade
        shortest_length = 0
        shortest_shade_weighted = 0
        shade_col = f'shade_{scenario_name}'
        
        for i in range(len(shortest_path) - 1):
            u = shortest_path[i]
            v = shortest_path[i + 1]
            
            # Get edge data (handle MultiDiGraph - take first edge)
            edge_data = graph[u][v][0]  # Key is 0 for first/only edge
            
            edge_length = edge_data['length']
            edge_shade = edge_data.get(shade_col, 0)
            
            shortest_length += edge_length
            shortest_shade_weighted += edge_shade * edge_length
        
        shortest_shade = shortest_shade_weighted / shortest_length if shortest_length > 0 else 0
        
        results['shortest_path'] = shortest_path
        results['shortest_length_ft'] = shortest_length
        results['shortest_length_m'] = shortest_length * 0.3048
        results['shortest_shade'] = shortest_shade
        
    except nx.NetworkXNoPath:
        if verbose:
            print(f"⚠️  Could not find shortest path")
        return None
    except Exception as e:
        if verbose:
            print(f"⚠️  Error in shortest path: {e}")
        return None
    
    # Calculate SHADIEST route (using time-specific weights)
    try:
        weight_col = f'weight_{scenario_name}'
        shadiest_path = nx.shortest_path(graph, origin_node, dest_node, weight=weight_col)
        
        # Calculate total length and weighted shade
        shadiest_length = 0
        shadiest_shade_weighted = 0
        
        for i in range(len(shadiest_path) - 1):
            u = shadiest_path[i]
            v = shadiest_path[i + 1]
            
            # Get edge data (handle MultiDiGraph)
            edge_data = graph[u][v][0]
            
            edge_length = edge_data['length']
            edge_shade = edge_data.get(shade_col, 0)
            
            shadiest_length += edge_length
            shadiest_shade_weighted += edge_shade * edge_length
        
        shadiest_shade = shadiest_shade_weighted / shadiest_length if shadiest_length > 0 else 0
        
        results['shadiest_path'] = shadiest_path
        results['shadiest_length_ft'] = shadiest_length
        results['shadiest_length_m'] = shadiest_length * 0.3048
        results['shadiest_shade'] = shadiest_shade
        
    except nx.NetworkXNoPath:
        if verbose:
            print(f"⚠️  Could not find shadiest path")
        return None
    except Exception as e:
        if verbose:
            print(f"⚠️  Error in shadiest path: {e}")
        return None
    
    # Comparison metrics
    results['length_increase_m'] = (shadiest_length - shortest_length) * 0.3048
    results['length_increase_pct'] = ((shadiest_length - shortest_length) / shortest_length * 100) if shortest_length > 0 else 0
    results['shade_improvement'] = shadiest_shade - shortest_shade
    results['shade_improvement_pct'] = ((shadiest_shade - shortest_shade) / shortest_shade * 100) if shortest_shade > 0 else 0
    
    if results['length_increase_m'] > 0:
        results['efficiency'] = results['shade_improvement'] / results['length_increase_m']
    else:
        results['efficiency'] = float('inf')
    
    if verbose:
        print(f"  {scenario_name}: {dest_stop_name}")
        print(f"    Shortest: {results['shortest_length_m']:.0f}m (shade: {shortest_shade:.3f})")
        print(f"    Shadiest: {results['shadiest_length_m']:.0f}m (shade: {shadiest_shade:.3f})")
        print(f"    Detour: +{results['length_increase_m']:.0f}m (+{results['length_increase_pct']:.1f}%)")
        print(f"    Shade gain: +{results['shade_improvement']:.3f}")
    
    return results

print("✓ Time-specific routing function defined (FIXED VERSION)")

✓ Time-specific routing function defined (FIXED VERSION)


## Part 3: Define Example Routes and Scenarios

In [28]:
# Define example routes to analyze
EXAMPLE_ROUTES = [
    {
        'name': 'Penn to 40th St',
        'origin': (39.9522, -75.1950),  # Penn campus (Locust Walk)
        'destination': '40th St' 
    },
    {
        'name': 'Drexel to 34th St',
        'origin': (39.9570, -75.1890),  # Drexel campus
        'destination': '34th St'
    },
    {
        'name': 'Clark Park to 46th St',
        'origin': (39.9480, -75.2080),  # Clark Park area
        'destination': '46th St'
    },
]

# Analysis times (from Notebook 2)
ANALYSIS_TIMES = {
    'summer_morning': 'Summer 8 AM',
    'summer_midday': 'Summer 12 PM',
    'summer_evening': 'Summer 5 PM',
    'winter_morning': 'Winter 8 AM',
    'winter_midday': 'Winter 12 PM',
    'winter_evening': 'Winter 5 PM',
    'spring_midday': 'Spring 12 PM',
    'fall_midday': 'Fall 12 PM',
}

print(f"✓ Defined {len(EXAMPLE_ROUTES)} example routes")
print(f"✓ Analyzing {len(ANALYSIS_TIMES)} time scenarios")
print(f"\nTotal route calculations: {len(EXAMPLE_ROUTES) * len(ANALYSIS_TIMES)} routes")

✓ Defined 3 example routes
✓ Analyzing 8 time scenarios

Total route calculations: 24 routes


## Part 4: Calculate Routes for All Scenarios

In [29]:
print("="*70)
print("CALCULATING TIME-OF-DAY ROUTES")
print("="*70)
print("\nThis will calculate routes for all time scenarios...\n")

all_routes = []

for route_def in EXAMPLE_ROUTES:
    print(f"\n{'='*70}")
    print(f"ROUTE: {route_def['name']}")
    print(f"{'='*70}")
    
    for scenario_name in ANALYSIS_TIMES.keys():
        route_result = calculate_time_specific_route(
            route_def['origin'],
            route_def['destination'],
            scenario_name,
            G,
            septa_gdf,
            verbose=False
        )
        
        if route_result:
            route_result['route_name'] = route_def['name']
            all_routes.append(route_result)
    
    print(f"  ✓ Calculated {len(ANALYSIS_TIMES)} scenarios for {route_def['name']}")

# Convert to DataFrame
routes_df = pd.DataFrame(all_routes)
print(f"\n" + "="*70)
print(f"✓ Total route calculations: {len(routes_df)}")
print("="*70)

# Save results
routes_df.to_csv('data/processed/time_of_day_routes.csv', index=False)
print(f"\n✓ Saved: data/processed/time_of_day_routes.csv")

CALCULATING TIME-OF-DAY ROUTES

This will calculate routes for all time scenarios...


ROUTE: Penn to 40th St
  ✓ Calculated 8 scenarios for Penn to 40th St

ROUTE: Drexel to 34th St
  ✓ Calculated 8 scenarios for Drexel to 34th St

ROUTE: Clark Park to 46th St
  ✓ Calculated 8 scenarios for Clark Park to 46th St

✓ Total route calculations: 0

✓ Saved: data/processed/time_of_day_routes.csv


## Part 5: Seasonal Pattern Analysis

In [30]:
def analyze_seasonal_patterns(routes_df):
    """
    Analyze how routing performance varies by season.
    """
    # Add season column
    def get_season(scenario):
        if 'summer' in scenario:
            return 'Summer'
        elif 'winter' in scenario:
            return 'Winter'
        elif 'spring' in scenario:
            return 'Spring'
        elif 'fall' in scenario:
            return 'Fall'
        return 'Unknown'
    
    routes_df['season'] = routes_df['scenario'].apply(get_season)
    
    # Add time of day
    def get_time_of_day(scenario):
        if 'morning' in scenario:
            return 'Morning (8 AM)'
        elif 'midday' in scenario:
            return 'Midday (12 PM)'
        elif 'evening' in scenario:
            return 'Evening (5 PM)'
        return 'Unknown'
    
    routes_df['time_of_day'] = routes_df['scenario'].apply(get_time_of_day)
    
    print("\n" + "="*70)
    print("SEASONAL PATTERN ANALYSIS")
    print("="*70)
    
    # Group by season
    seasonal_summary = routes_df.groupby('season').agg({
        'shortest_shade': 'mean',
        'shadiest_shade': 'mean',
        'shade_improvement': 'mean',
        'length_increase_pct': 'mean',
        'efficiency': 'mean'
    }).round(3)
    
    print("\nSeasonal Performance (all routes averaged):")
    print(seasonal_summary)
    
    # Time of day patterns
    time_summary = routes_df.groupby('time_of_day').agg({
        'shortest_shade': 'mean',
        'shadiest_shade': 'mean',
        'shade_improvement': 'mean',
    }).round(3)
    
    print("\nTime-of-Day Patterns:")
    print(time_summary)
    
    # Best and worst scenarios
    print("\n" + "-"*70)
    print("BEST SCENARIOS (highest shade improvement):")
    best = routes_df.nlargest(5, 'shade_improvement')[
        ['route_name', 'scenario', 'shade_improvement', 'length_increase_pct']
    ]
    print(best.to_string(index=False))
    
    print("\nWORST SCENARIOS (lowest shade improvement):")
    worst = routes_df.nsmallest(5, 'shade_improvement')[
        ['route_name', 'scenario', 'shade_improvement', 'length_increase_pct']
    ]
    print(worst.to_string(index=False))
    
    return routes_df, seasonal_summary, time_summary

# Run analysis
routes_df_analyzed, seasonal_summary, time_summary = analyze_seasonal_patterns(routes_df)

KeyError: 'scenario'

## Part 6: Comprehensive Comparison Visualizations

In [ ]:
# Create comprehensive comparison figure
fig = plt.figure(figsize=(18, 12))

# 1. Shade by Season (Box Plot)
ax1 = plt.subplot(2, 3, 1)
routes_df_analyzed.boxplot(column='shadiest_shade', by='season', ax=ax1)
ax1.set_title('Shade Coverage by Season', fontsize=14, fontweight='bold')
ax1.set_xlabel('Season', fontsize=12)
ax1.set_ylabel('Shade Score (0-1)', fontsize=12)
plt.sca(ax1)
plt.xticks(rotation=0)
plt.suptitle('')

# 2. Shade by Time of Day
ax2 = plt.subplot(2, 3, 2)
routes_df_analyzed.boxplot(column='shadiest_shade', by='time_of_day', ax=ax2)
ax2.set_title('Shade Coverage by Time of Day', fontsize=14, fontweight='bold')
ax2.set_xlabel('Time of Day', fontsize=12)
ax2.set_ylabel('Shade Score (0-1)', fontsize=12)
plt.sca(ax2)
plt.xticks(rotation=45, ha='right')
plt.suptitle('')

# 3. Detour vs Shade Improvement (Scatter)
ax3 = plt.subplot(2, 3, 3)
for season in routes_df_analyzed['season'].unique():
    season_data = routes_df_analyzed[routes_df_analyzed['season'] == season]
    ax3.scatter(season_data['length_increase_pct'], 
               season_data['shade_improvement'],
               label=season, alpha=0.6, s=100)
ax3.set_title('Trade-off: Detour vs Shade Benefit', fontsize=14, fontweight='bold')
ax3.set_xlabel('Length Increase (%)', fontsize=12)
ax3.set_ylabel('Shade Improvement', fontsize=12)
ax3.legend()
ax3.grid(alpha=0.3)

# 4. Seasonal Comparison (Bar)
ax4 = plt.subplot(2, 3, 4)
seasonal_summary[['shortest_shade', 'shadiest_shade']].plot(kind='bar', ax=ax4)
ax4.set_title('Shortest vs Shadiest Routes by Season', fontsize=14, fontweight='bold')
ax4.set_xlabel('Season', fontsize=12)
ax4.set_ylabel('Mean Shade Score', fontsize=12)
ax4.legend(['Shortest Route', 'Shadiest Route'])
plt.sca(ax4)
plt.xticks(rotation=0)

# 5. Time-of-Day Heatmap
ax5 = plt.subplot(2, 3, 5)
pivot_data = routes_df_analyzed.pivot_table(
    values='shadiest_shade',
    index='season',
    columns='time_of_day',
    aggfunc='mean'
)
sns.heatmap(pivot_data, annot=True, fmt='.3f', cmap='YlGn', ax=ax5,
            cbar_kws={'label': 'Shade Score'})
ax5.set_title('Shade Coverage: Season × Time of Day', fontsize=14, fontweight='bold')
ax5.set_xlabel('Time of Day', fontsize=12)
ax5.set_ylabel('Season', fontsize=12)

# 6. Efficiency by Scenario
ax6 = plt.subplot(2, 3, 6)
efficiency_data = routes_df_analyzed.groupby('scenario')['efficiency'].mean().sort_values(ascending=False)
efficiency_data = efficiency_data[efficiency_data != float('inf')]
efficiency_data.plot(kind='barh', ax=ax6, color='steelblue')
ax6.set_title('Routing Efficiency by Scenario', fontsize=14, fontweight='bold')
ax6.set_xlabel('Efficiency (shade per meter detour)', fontsize=12)
ax6.set_ylabel('Scenario', fontsize=12)

plt.tight_layout()
plt.savefig('outputs/figures/time_of_day_comprehensive_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Saved: outputs/figures/time_of_day_comprehensive_analysis.png")

## Part 7: Route-Specific Analysis

In [ ]:
# Create detailed comparison for each route
for route_name in routes_df_analyzed['route_name'].unique():
    route_data = routes_df_analyzed[routes_df_analyzed['route_name'] == route_name]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'{route_name}: Time-of-Day & Seasonal Analysis', 
                 fontsize=16, fontweight='bold')
    
    # 1. Shade coverage across scenarios
    ax1 = axes[0, 0]
    x_pos = np.arange(len(route_data))
    ax1.bar(x_pos, route_data['shortest_shade'], alpha=0.7, 
           label='Shortest Route', color='coral')
    ax1.bar(x_pos, route_data['shadiest_shade'], alpha=0.7,
           label='Shadiest Route', color='forestgreen')
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels(route_data['scenario'], rotation=45, ha='right')
    ax1.set_ylabel('Shade Score', fontsize=12)
    ax1.set_title('Shade Coverage Comparison', fontsize=13, fontweight='bold')
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    # 2. Length increase
    ax2 = axes[0, 1]
    ax2.bar(x_pos, route_data['length_increase_m'], color='steelblue', alpha=0.7)
    ax2.set_xticks(x_pos)
    ax2.set_xticklabels(route_data['scenario'], rotation=45, ha='right')
    ax2.set_ylabel('Additional Distance (meters)', fontsize=12)
    ax2.set_title('Detour Required for Shade', fontsize=13, fontweight='bold')
    ax2.grid(axis='y', alpha=0.3)
    
    # 3. Shade improvement
    ax3 = axes[1, 0]
    colors = ['darkgreen' if x > 0.15 else 'orange' if x > 0.10 else 'gray' 
              for x in route_data['shade_improvement']]
    ax3.bar(x_pos, route_data['shade_improvement'], color=colors, alpha=0.7)
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels(route_data['scenario'], rotation=45, ha='right')
    ax3.set_ylabel('Shade Improvement', fontsize=12)
    ax3.set_title('Benefit from Optimization', fontsize=13, fontweight='bold')
    ax3.axhline(y=0.15, color='green', linestyle='--', alpha=0.5, label='High benefit')
    ax3.axhline(y=0.10, color='orange', linestyle='--', alpha=0.5, label='Moderate benefit')
    ax3.legend()
    ax3.grid(axis='y', alpha=0.3)
    
    # 4. Efficiency
    ax4 = axes[1, 1]
    efficiency_finite = route_data['efficiency'].replace([np.inf, -np.inf], np.nan)
    ax4.bar(x_pos, efficiency_finite, color='purple', alpha=0.7)
    ax4.set_xticks(x_pos)
    ax4.set_xticklabels(route_data['scenario'], rotation=45, ha='right')
    ax4.set_ylabel('Efficiency (shade/meter)', fontsize=12)
    ax4.set_title('Routing Efficiency', fontsize=13, fontweight='bold')
    ax4.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    
    # Save
    filename = route_name.lower().replace(' ', '_')
    plt.savefig(f'outputs/figures/{filename}_seasonal_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✓ Saved: outputs/figures/{filename}_seasonal_analysis.png")

## Part 8: Generate Summary Statistics

In [ ]:
# Create comprehensive summary table
summary_table = routes_df_analyzed.groupby(['season', 'time_of_day']).agg({
    'shortest_shade': ['mean', 'std'],
    'shadiest_shade': ['mean', 'std'],
    'shade_improvement': ['mean', 'std'],
    'length_increase_pct': ['mean', 'std'],
    'efficiency': 'mean'
}).round(3)

print("\n" + "="*80)
print("COMPREHENSIVE SUMMARY: SEASON × TIME OF DAY")
print("="*80)
print(summary_table)

# Save to CSV
summary_table.to_csv('data/processed/seasonal_time_summary.csv')
print("\n✓ Saved: data/processed/seasonal_time_summary.csv")

# Key findings
print("\n" + "="*80)
print("KEY FINDINGS")
print("="*80)
print(f"\n📊 Overall Statistics:")
print(f"  Average detour: {routes_df_analyzed['length_increase_pct'].mean():.1f}%")
print(f"  Average shade improvement: {routes_df_analyzed['shade_improvement'].mean():.3f}")
print(f"  Best season: {seasonal_summary['shadiest_shade'].idxmax()}")
print(f"  Worst season: {seasonal_summary['shadiest_shade'].idxmin()}")
print(f"  Best time: {time_summary['shadiest_shade'].idxmax()}")

print("\n🌡️ Seasonal Comparison:")
for season in seasonal_summary.index:
    shade = seasonal_summary.loc[season, 'shadiest_shade']
    detour = seasonal_summary.loc[season, 'length_increase_pct']
    print(f"  {season:10s}: shade={shade:.3f}, detour={detour:.1f}%")

print("\n⏰ Time-of-Day Comparison:")
for time in time_summary.index:
    shade = time_summary.loc[time, 'shadiest_shade']
    print(f"  {time:20s}: shade={shade:.3f}")

In [ ]:
# Final summary- Notebook 3

print("\nAnalysis Completed:")
print(f"  Routes analyzed: {len(EXAMPLE_ROUTES)}")
print(f"  Time scenarios: {len(ANALYSIS_TIMES)}")
print(f"  Total calculations: {len(routes_df_analyzed)}")

print("\nFiles Created:")
output_files = [
    'data/processed/time_of_day_routes.csv',
    'data/processed/seasonal_time_summary.csv',
    'outputs/figures/time_of_day_comprehensive_analysis.png',
]
for f in output_files:
    if os.path.exists(f):
        print(f"  ✓ {f}")

route_figures = [f'outputs/figures/{r["name"].lower().replace(" ", "_")}_seasonal_analysis.png' 
                for r in EXAMPLE_ROUTES]
for f in route_figures:
    if os.path.exists(f):
        print(f"  ✓ {f}")